# **SET UP**

## **Install**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/BIOINFORMATICS_LECTURE/SD4108/PERTEMUAN/COLAB_version')

In [ ]:
! pip install Biopython
! sudo apt-get install -y clustalw
#! chmod +x clustalw2

## **Import**

In [ ]:
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Align.Applications import ClustalwCommandline
import os

# **0. FETCH FASTA SEQUENCE FROM PUBLIC DATABASE (NCBI, GENBANK, GISAID)**

## **0A. Fetching a sequence (ID) from GenBank/NCBI using biopython**

In [ ]:
from Bio import Entrez, SeqIO

# input email sebagai identitas
Entrez.email = "your-email@example.com"

# Fetch sekuen dengan menggunakan ACCESSION NUMBER ID GENBANK
def fetch_sequence_from_genbank(accession_id):
    handle = Entrez.efetch(db="nucleotide", id=accession_id, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()

    # Print the sequence
    print(f"Description: {record.description}")
    print(f"Sequence: {record.seq}")
    #print(f"Sequence: {record.annotations}")

    return record

# Example usage
genbank_record = fetch_sequence_from_genbank("AB425198.1")  # Example: esophageal cancer protein 1 genome

### check hieracycal data structure

In [ ]:
# annotations field; dic that provides a number of properties for the seq
print('Annotations dictionary:\n')
print(genbank_record.annotations)

print('\nKeys:')
print(genbank_record.annotations.keys())

In [ ]:
# Getting specific key values
gb=genbank_record
print('\nGet specific parts of the annotation:\n')
print('Taxonomy:\n')
print(gb.annotations['taxonomy'])

print('Source:\n')
print(gb.annotations['source'])

print('Date:\n')
print(gb.annotations['date'])

#print('gi number:\n')
#print(gb.annotations['gi'])

In [ ]:
gb.features[:5] # show only a few features (total=41)

In [ ]:
gene_features = []
for i in range(len(gb.features)):
    if(gb.features[i].type == 'gene'):
        gene_features.append(gb.features[i])

print(f'Number of gene features: {len(gene_features)}')
gene_features

In [ ]:
# Gene Qualifiers
gene_features[0].qualifiers

In [ ]:
CDS_features = []
for i in range(len(gb.features)):
    if(gb.features[i].type == 'CDS'):
        CDS_features.append(gb.features[i])

print(f"Number of CDS features: {len(CDS_features)}")
CDS_features

In [ ]:
print(f'CDS Qualifier Keys: {CDS_features[0].qualifiers.keys()}\n')

print('Showing First CDS Feature')
print(CDS_features[0].qualifiers) # ordered dictionary

In [ ]:
for key, value in CDS_features[0].qualifiers.items():
    print(f'{key} : {value}')

## **0B. Fetching Multiple sequences by query from NCBI**

In [ ]:
from Bio import Entrez, SeqIO

Entrez.email = "your-email@example.com"

# Function to fetch multiple sequences from GenBank based on a search query
def fetch_multiple_sequences(query, max_records):

    # Search for nucleotide sequences matching the query
    search_handle = Entrez.esearch(db="nucleotide", term=query, retmax=max_records)
    search_results = Entrez.read(search_handle)
    search_handle.close()

    ids = search_results["IdList"]

    # Fetch the sequences in FASTA format
    fetch_handle = Entrez.efetch(db="nucleotide", id=ids, rettype="fasta", retmode="text")
    sequences = list(SeqIO.parse(fetch_handle, "fasta"))
    fetch_handle.close()

    # Print the total number of sequences fetched
    print(f"Fetched {len(sequences)} sequences")

    # Optionally save to a file
    with open("sequences.fasta", "w") as output_handle:
        SeqIO.write(sequences, output_handle, "fasta")

    return sequences

# Example usage: Search for BRCA1 gene in Homo sapiens
sequences = fetch_multiple_sequences("BRCA1[Gene] AND Homo sapiens[Organism]", max_records=50)

## **0C. Reading sequences from local FASTA file**

In [ ]:
from Bio import SeqIO

# Function to read sequences from a FASTA file
def read_fasta(file_path):
    sequences = list(SeqIO.parse(file_path, "fasta"))
    print(f"Found {len(sequences)} sequences in {file_path}")

    for seq_record in sequences[:5]:  # Print the first 5 sequences
        print(f"ID: {seq_record.id}")
        print(f"Sequence: {seq_record.seq[:100]}...")  # Print the first 100 bases

    return sequences

# Example usage
fasta_sequences = read_fasta("sequences.fasta")

## **0D. Akses NCBI Virus Data Hub**

In [ ]:
def fetch_viral_sequences(query, max_records):
    Entrez.email = "your-email@example.com"

    # Search for viral sequences
    search_handle = Entrez.esearch(db="nucleotide", term=query, retmax=max_records)
    search_results = Entrez.read(search_handle)
    search_handle.close()

    ids = search_results["IdList"]

    # Fetch the sequences in FASTA format
    fetch_handle = Entrez.efetch(db="nucleotide", id=ids, rettype="fasta", retmode="text")
    sequences = list(SeqIO.parse(fetch_handle, "fasta"))
    fetch_handle.close()

    # Save to a file
    with open("viral_sequences.fasta", "w") as output_handle:
        SeqIO.write(sequences, output_handle, "fasta")

    return sequences

# Example usage: Fetching SARS-CoV-2 sequences
viral_sequences = fetch_viral_sequences("SARS-CoV-2[Organism]", max_records=100)

## **0E. Downloading Data in Bulk with Biophyton**

In [ ]:
# Download a large number of records (e.g., 1000 sequences) in batches of 200
def batch_download_sequences(query, batch_size=200, total_records=1000):
    Entrez.email = "your-email@example.com"

    for start in range(0, total_records, batch_size):
        handle = Entrez.esearch(db="nucleotide", term=query, retstart=start, retmax=batch_size)
        record = Entrez.read(handle)
        ids = record["IdList"]

        # Fetch sequences
        handle = Entrez.efetch(db="nucleotide", id=ids, rettype="fasta", retmode="text")
        data = handle.read()
        handle.close()

        with open(f"sequences_batch_{start}.fasta", "w") as f:
            f.write(data)

        print(f"Downloaded {len(ids)} records (Batch {start // batch_size + 1})")

# Example: Downloading large viral dataset
batch_download_sequences("SARS-CoV-2[Organism]", batch_size=200, total_records=1000)

## **0F. Downloading Data in Bulk with Biophyton using reference file (.txt)**

In [ ]:
def download_cytochrome_b_sequences(species_list):
    Entrez.email='researchtirta@gmail.com'

    for species in species_list:
        search_term = f"{species} AND mitochondrial cytochrome b gene, complete cds AND 1:2000[SLEN]"
        handle = Entrez.esearch(db='nucleotide', term=search_term, retmax=5)
        record = Entrez.read(handle)
        handle.close()

        if int(record['Count'] == 0):
            print(f"No cytochrome b sequences found for {species}")
        else:
            seq_ids = record['IdList']
            handle = Entrez.efetch(db='nucleotide', id=seq_ids, rettype='fasta', retmode='text')
            sequences = SeqIO.parse(handle,'fasta')
            sequence_list = list(sequences)

            output_file = f"{species}_cytb.fasta"
            SeqIO.write(sequence_list, output_file, 'fasta')
            handle.close()
            num_sequences = len(sequence_list)
            print(f"Downloaded {num_sequences} cytochrome b sequence(s) for {species}")

In [ ]:
file_path = 'species_file.txt'

with open(file_path, 'r') as file:
    species_list = [line.strip() for line in file]

download_cytochrome_b_sequences(species_list)

## **0G. Combine or merge multiple fasta file**

In [ ]:
from Bio import SeqIO

# List of input FASTA files you want to combine
fasta_files = ["sequences_batch_0.fasta", "sequences_batch_200.fasta",
               "sequences_batch_400.fasta", "sequences_batch_600.fasta",
               "sequences_batch_800.fasta"]

# Name of the output file
output_file = "combined_sequences.fasta"

# Open the output file for writing
with open(output_file, "w") as outfile:
    for fasta_file in fasta_files:
        # Parse each input FASTA file
        with open(fasta_file, "r") as infile:
            # Use SeqIO.parse to read sequences and write them to the output file
            for record in SeqIO.parse(infile, "fasta"):
                SeqIO.write(record, outfile, "fasta")

print(f"Sequences from {len(fasta_files)} files have been combined into {output_file}")

## **LATIHAN**

### Tugas

1. Silahkan buat analisis sekuen dengan ID: NC_045512:

    a. Temukan Key dan Value dari ID tersebut

    b. Jelaskan informasi terkait sekuen dengan ID tersebut

2. Downloadlah dataset variant dari covid-19 informasi ada disini https://en.wikipedia.org/wiki/Variants_of_SARS-CoV-2:

    a. Downloadlah menggunakan file referensi dengan format .txt

    b. Gunakan fungsi untuk melakukan merger file dengan format **(fasta dan ganbank)**


3. Tugas dikumpulkan dalam format nim_nama.ipynb (untuk kodingan) dan nim_nama.fasta & nim_nama.gb pada GCR sebelum 1 Oktober 2024, pukul 14.30
